In [2]:
using LinearAlgebra, CSV, Random, Tables, JLD, Base.Iterators, Printf

In [3]:
include("../Algorithm/utilities.jl")

refreshFile (generic function with 1 method)

In [4]:
data = load("../Data/communities.jld")["communities"]
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
norms = data[1:n+1:N]
k = 5;
# All close.
maximum(abs.(raw_data * raw_data .- data))

7.494005416219807e-16

In [5]:
nodes = CSV.File("../Co-K5-Trace-Nodes.csv", header=false) |> Tables.matrix;
bounds = CSV.File("../Co-K5-Trace-Bounds.csv", header=false) |> Tables.matrix;

In [6]:
# node_id = 50
# node_id = 300
node_id = 40
y = nodes[:, node_id]
selected_data = findall(y .== 1)
i = sum(nodes[:, node_id] .== 1)
stillneed = k - i

2

In [7]:
bounds[:, node_id]

3-element Vector{Float64}:
 0.19840666121229678
 0.3629025164807017
 0.4316969652057027

In [8]:
sum(norms[y .== 1]) + sum(selectsorted(norms .* (y .== -1), stillneed))

0.4316969652057027

In [9]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s^2

0.14185918642419365

In [10]:
norm(raw_data[:, selected_data] * v)^2

0.14185918642419362

In [11]:
norm(u' * raw_data[:, selected_data])^2

0.14185918642419373

In [12]:
norm(data[:, selected_data][selected_data, :] * v)

0.1418591864241938

In [13]:
rank_one_term = (raw_data * u) .^ 2
rank_one_term[nodes[:, node_id] .== 0] .= 0
rank_one_term

101-element Vector{Float64}:
 7.926078166037313e-6
 0.0004210751967121292
 0.010091876290498321
 0.0
 0.011482437882389973
 0.00013178020948110262
 0.0023632781250717753
 0.0009055802857870328
 0.0012990605554894894
 0.0016750788941365191
 6.978641405514641e-6
 0.038350149020200563
 0.0
 ⋮
 3.042644920930742e-5
 5.898083857838683e-7
 0.009444173176467858
 0.006324633814033148
 0.00029115092252889356
 0.0005201627269077607
 0.0008469155188951948
 3.4017703642097506e-5
 0.001632714485715438
 0.005953773016861566
 0.0001847886202275514
 0.006647989769474606

In [14]:
# Upper bound on the contribution from adding one variable into ||E||^2.
# The upper bound comes from ||E||_F^2
# upper bounded by ||(I-uu')MD||_F^2. Also, we know that normCommunities
# has column norm of 1 for every variable.
# We should add another upper bound based on taking k columns from this
# row into ||E||_F.
residual_term = norms .- rank_one_term
residual_term[y .== 0] .= 0
residual_term

101-element Vector{Float64}:
 0.016097164406534425
 0.026382093189530394
 0.05414100146972663
 0.0
 0.03214735697131309
 0.05392057454670921
 0.021722436645934658
 0.01970490065528001
 0.026424705247690236
 0.030432363176178798
 0.01644266190089374
 0.1595062684006523
 0.0
 ⋮
 0.010497707475897677
 0.010079594849954536
 0.04397882892064134
 0.03542551701477037
 0.03259738143319357
 0.03968867072276389
 0.038444722591623
 0.011947288809978045
 0.03961357916191083
 0.04651232744206663
 0.05757289255173794
 0.04763397466371457

In [15]:
function frobenius_rows_k(M, y)
    M = M .^ 2
    row_quantities = zeros(n)
    stillneed = k - sum(y .== 1)
    for row_index in 1:n
        if y[row_index] == 0
            continue
        end
        row = copy(M[row_index, :])
        row_sum = sum(row[y .== 1])
        row[y .== 1] .= 0
        if y[row_index] == -1
            row_sum += row[row_index]
            row[row_index] = 0
            row_stillneed = stillneed -1
        else
            row_stillneed = stillneed
        end
        row_sum += sum(sort(row, rev=true)[1:row_stillneed])
        row_quantities[row_index] = row_sum
    end
    row_quantities
end

frobenius_rows_k (generic function with 1 method)

In [16]:
# frobenius_rows_k(data - raw_data * u * u' * raw_data, y)

In [17]:
sum((y .== -1) .& (frobenius_rows_k(data - raw_data * u * u' * raw_data, y) .< residual_term))

81

In [18]:
lb_proj_nodes_extra = rank_one_term .>= sort(rank_one_term .* Array{Float64}(y .== -1))[end-stillneed+1]
lb_proj_nodes_extra = lb_proj_nodes_extra .& (y .== -1)
lb_proj_nodes = lb_proj_nodes_extra .| (y .== 1)
lb_proj = sum(rank_one_term[lb_proj_nodes])
# Too slow to compute!
lb_diag = maximum(svd(raw_data[:, lb_proj_nodes]).S) ^ 2
lb_proj, lb_diag

(0.2202679971348655, 0.2732082054995252)

In [19]:
bounds[:, node_id]

3-element Vector{Float64}:
 0.19840666121229678
 0.3629025164807017
 0.4316969652057027

In [20]:
lambda_1 = bounds[2, node_id]
lambda_2 = bounds[3, node_id] - lb_diag
lambda_2_lb = bounds[3, node_id] - lambda_1
lambda_2_lb, lambda_2

(0.06879444872500101, 0.15848875970617754)

In [21]:
# lambda_2 = svd(raw_data[:, selected_data]).S[2] + sum(selectsorted(residual_term, stillneed))

In [22]:
# Initialization. Loop until convergence.
# Bad idea - lambda_2 bounds ||E|| from below, not above.
residual_ub = lambda_2
# Contribution to Tr E < residual_ub
resid_var_sel = residual_term .<= residual_ub
resid_var_sel = BitArray(ones(size(y)))
residual_ub, sum((y .== -1) .& .~resid_var_sel)

(0.15848875970617754, 0)

In [23]:
# ||Sigma||_F >= sqrt(||A||_F + ||E||_F)???
norm(data[1:k, 1:k]), sqrt(norm(raw_data[1:k, :] * u * u' * raw_data[:, 1:k])^2 + norm(data[1:k, 1:k] - (raw_data[1:k, :] * u * u' * raw_data[:, 1:k]))^2)

(0.1282440427818185, 0.11760011772186946)

In [24]:
a_squared_program = mapslices(
    (row) -> sum(row[y .== 1].^2) + sum(selectsorted(row.^2 .* (y .== -1), stillneed)),
    data,
    dims=2)[:, 1]
# lambda_1^2 >= sum_i rank_one_term_i^2 >= sum_i rank_one_term_i^2 + sum_(i!=j) 2 rank_one_term_i rank_one_term_j
e_squared_program = a_squared_program - rank_one_term.^2
maximum(e_squared_program)

0.04451876283731441

In [25]:
# Contribution to ||E||^2 from the row sums (squared Frobenius norm)
E = data - (raw_data * u * u' * raw_data)
e_squared_program = mapslices(
    (row) -> sum(row[y .== 1].^2) + sum(selectsorted(row.^2 .* (resid_var_sel .& (y .== -1)), stillneed)),
    E,
    dims=2)[:, 1]
maximum(e_squared_program)

0.026386179743713503

In [26]:
# Upper bound on ||E||_F using ||A||_F
sqrt(sum(a_squared_program[y .== 1]) + sum(selectsorted(a_squared_program .* (y .== -1), stillneed)) - lb_diag^2)

0.12443300672604672

In [27]:
# Upper bound on ||E||_F
lambda_2 = sqrt(sum(e_squared_program[y .== 1]) + sum(selectsorted(e_squared_program .* (y .== -1), stillneed)))

0.18117842668099743

In [28]:
# Upper bound performed poorly
lambda_2 = bounds[3, node_id] - lb_diag

0.15848875970617754

In [29]:
sum(residual_term[y .== 1]) + sum(selectsorted(residual_term[y .== -1], stillneed))

0.24795709056878376

In [30]:
# Multiplier for 
mult_parallel = 1. / lambda_1
mult_perp = lambda_2 / (lb_proj - lambda_2)^2
mult_linearize = mult_parallel + mult_perp
linear_program = rank_one_term + mult_linearize * e_squared_program

101-element Vector{Float64}:
 0.024601676904719726
 0.05799532039097986
 0.18262281400917188
 0.14606029764996195
 0.0880375464012855
 0.23925642932289726
 0.04348631750007593
 0.03823110015959428
 0.054801606237631605
 0.07411834186327425
 0.028331367049840104
 1.206754422470488
 0.014195255911639011
 ⋮
 0.011018768210548377
 0.008226682834820325
 0.1926588984214859
 0.10085579242512233
 0.0871036705508794
 0.10924901820611166
 0.10349046799096258
 0.012162377026588875
 0.10449036260751617
 0.12937540070461656
 0.18853743075691767
 0.15193791934247905

In [31]:
# Linear program solution
sum(linear_program[y .== 1]) + sum(selectsorted(linear_program[y .== -1], stillneed))

1.6338898195663933

In [36]:
# Conservative program in case we cannot bound sin theta_perp.
linear_program_lam_2 = rank_one_term + mult_parallel .* e_squared_program
obj = lambda_2 + sum(linear_program_lam_2[y .== 1]) + sum(selectsorted(linear_program_lam_2[y .== -1], stillneed))

0.45466491135441633

In [49]:
# (||B'B||_F + ||C'C||_F)^2 = (C11^2 + 2 C11 C22 + C22^2) + 2sqrt(C11^2 + 2 C11 C22 + C22^2) B11 + B11^2
# ||B'B + C'C||_F^2 = (C11 + B11)^2 + 2 C11 C22 + C22^2
selected = [findall(y .== 1); sortperm(linear_program_lam_2 .* (y .== -1), rev=true)[1:stillneed]]
sqrt(obj^2 + 2 * sum(rank_one_term[selected]) * (mult_parallel - mult_parallel - mult_perp) * sum(e_squared_program[selected]))

LoadError: DomainError with -0.297213273627568:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [47]:
obj^2, sum(rank_one_term[selected])

(0.20672018161691927, 0.2202679971348655)

In [43]:
(sqrt(mult_parallel^2 + mult_perp^2) - mult_parallel) * sum(e_squared_program[selected])

1.070517487176781

In [ ]:
[sum((sort(rank_one_term, rev=true)[k] .> linear_program) .& (y .== -1)) sum(y .== -1)]

1×2 Array{Int64,2}:
 13  81

In [ ]:
resid_var_sel = y .== -1
for iter in countfrom()
    @printf("rank_one_term iter %d\n", iter)
    e_squared_program = mapslices(
        (row) -> sum(row[y .== 1].^2) + sum(selectsorted(row.^2 .* (resid_var_sel .& (y .== -1)), stillneed)),
        E,
        dims=2)[:, 1]
    linear_program = rank_one_term + mult_linearize * e_squared_program
    num_selected = sum(resid_var_sel)
    resid_var_sel .&= sort(rank_one_term, rev=true)[k] .<= linear_program
    if sum(resid_var_sel) == num_selected
        break
    end
end

rank_one_term iter 1
rank_one_term iter 

2


In [ ]:
sum(resid_var_sel)

68

In [ ]:
# Linear program solution
sum(linear_program[y .== 1]) + sum(selectsorted(linear_program[y .== -1], stillneed))

1.6338898195663925